# Imports

In [1]:
#!pip install langdetect
#!pip install py-readability-metrics
#!pip install pyLDAvis
#!pip install pyngrok
#!pip install visdom
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from langdetect import detect
from readability import Readability
import gensim
import gensim.corpora as corpora
from gensim.models import Phrases
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from sklearn.feature_extraction.text import CountVectorizer
tqdm.pandas()

In [3]:
from pyngrok import conf, ngrok
import os

conf.get_default().config_path = "./ngrok/config.yml"

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(8097)
print(public_url)

NgrokTunnel: "https://4d6c-172-83-13-4.ngrok-free.app" -> "http://localhost:8097"


t=2023-09-06T23:13:28+0000 lvl=warn msg="failed to open private leg" id=d1e7271a238b privaddr=localhost:8097 err="dial tcp 127.0.0.1:8097: connect: connection refused"
t=2023-09-06T23:13:29+0000 lvl=warn msg="failed to open private leg" id=fb3043e3f010 privaddr=localhost:8097 err="dial tcp 127.0.0.1:8097: connect: connection refused"


# Path

In [9]:
main_path = 'D:/Uni/Bachelorthesis/'

# Data

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
df_kickstarter = pd.read_csv(main_path + 'kickstarter.csv')

# Functions

In [ ]:
def numWords(text):
    r = Readability(text)
    try:
        t = r.statistics()
        return t['num_words']
    except:
        return None

In [ ]:
def removeHTML(html_str):
    soup = BeautifulSoup(str(html_str), features="html.parser")
    return soup.get_text()

In [ ]:
def remove_urls(document):
    document = re.sub(r'http\S+', '', str(document))
    return document

In [ ]:
def remove_parting_lines(document):
    pattern = r'^[\*\-#]{5,}.+$'
    document = re.sub(pattern, '', str(document), flags=re.MULTILINE)
    return document

In [ ]:
def detectLang(t):
    try:
        return detect(t)
    except:
        return None

In [2]:
def preprocess_text(document: str, stemmer: nltk.stem.WordNetLemmatizer, en_stop: set) -> list:
    """Preprocesses a document to remove special characters/whitespace/etc

    Args:
        document (str):
        stemmer (nltk.stem.WordNetLemmatizer): Stemmer from NLTK
        en_stop (set): Set of stop words, usually from NLTK

    Returns:
        str: preprocessed document
    """

    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Tokenization
    tokens = nltk.word_tokenize(document)

    # POS-Tagging
    tagged_tokens = nltk.pos_tag(tokens)

    # Filter Nouns and Lemmatization
    lemmatized_nouns = []
    for token, pos in tagged_tokens:
        if pos.startswith('N'):  # Check if the token is a noun
            lemma = stemmer.lemmatize(token)
            # FIlter stop words, words that contain only numbers and short words
            if lemma not in en_stop and not lemma.isdigit() and len(lemma) > 2:
                lemmatized_nouns.append(lemma)

    return lemmatized_nouns;

In [3]:
def preprocess_text_helper(t):
    stemmer = nltk.stem.WordNetLemmatizer()
    return preprocess_text(t, stemmer, stopwords.words('english'))

# General Cleaning

In [ ]:
# Filter rows based on column: 'project_country'
df_kickstarter = df_kickstarter[df_kickstarter['project_country'] == "US"]

In [ ]:
# Filter rows based on column: 'project_currency'
df_kickstarter = df_kickstarter[df_kickstarter['project_currency'] == "USD"]

In [ ]:
# Drop rows with missing data in column: 'project_title'
df_kickstarter = df_kickstarter.dropna(subset=['project_title'])

In [ ]:
# Drop rows with missing data in column: 'project_description'
df_kickstarter = df_kickstarter.dropna(subset=['project_title'])

In [ ]:
# Drop duplicate rows in column: 'project_description'
df_kickstarter = df_kickstarter.drop_duplicates(subset=['project_description'])

In [ ]:
df_kickstarter['project_description'] = df_kickstarter['project_description'].apply(removeHTML)

In [ ]:
df_kickstarter['project_description'] = df_kickstarter['project_description'].apply(remove_urls)

In [ ]:
df_kickstarter['project_description'] = df_kickstarter['project_description'].apply(remove_parting_lines)

In [ ]:
nltk.download('punkt')
df_kickstarter['num_words'] = df_kickstarter['project_description'].progress_apply(numWords)
df_kickstarter = df_kickstarter[df_kickstarter['num_words'] >= 100]
# Reducing outliers (very long project descriptions might effect topic modelling negatively)
df_kickstarter = df_kickstarter[df_kickstarter['num_words'] <= 3000]

In [ ]:
df_kickstarter['project_lang'] = df_kickstarter['project_description'].progress_apply(detectLang)
df_kickstarter = df_kickstarter[df_kickstarter['project_lang'] == 'en']

# LDA

In [4]:
df_lda_kickstarter = pd.read_csv('./kickstarter_cleaned_topicmodelling.csv')

In [ ]:
# Copy Dataframe
# df_lda_kickstarter = df_kickstarter.copy()

In [5]:
# Remove the columns
df_lda_kickstarter.drop(df_lda_kickstarter.columns.difference(['project_description', 'project_category_id', 'project_parent_category_id', 'project_state']), axis=1,inplace=True)

### Technology Category

In [6]:
df_lda_kickstarter_technology = df_lda_kickstarter.copy()

In [7]:
# Filter Dataframe by Technology category (number 16)
df_lda_kickstarter_technology = df_lda_kickstarter_technology[(df_lda_kickstarter_technology['project_category_id'] == 16) | (df_lda_kickstarter_technology['project_parent_category_id'] == 16)]

In [8]:
# Preprocessing general steps
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
df_lda_kickstarter_technology['project_description'] = df_lda_kickstarter_technology['project_description'].progress_apply(preprocess_text_helper)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CoolerMaster\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CoolerMaster\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CoolerMaster\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


  0%|          | 0/26976 [00:00<?, ?it/s]

In [12]:
data_technology = df_lda_kickstarter_technology['project_description'].tolist()
# Compute n-grams
from gensim.models import Phrases

# Add n-grams to docs (only ones that appear 20 times or more).
ngrams_technology = Phrases(data_technology, min_count=20)
for idx in range(len(data_technology)):
    for token in ngrams_technology[data_technology[idx]]:
        if '_' in token:
            # Token is a n-grams, add to document.
            data_technology[idx].append(token)

In [13]:
# create dictionary
dictionary_technology = corpora.Dictionary(data_technology)

In [14]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_technology.filter_extremes(no_below=5, no_above=0.5)

In [15]:
# Define your custom stoplist
custom_stoplist_technology = ["project", "kickstarter", "pledge", "backer", "campaign", "product"] #to be extended
# Add your custom stop words to the dictionary
stop_ids_technology = [dictionary_technology.token2id[word] for word in custom_stoplist_technology if word in dictionary_technology.token2id]
# Remove the stop words from the dictionary
dictionary_technology.filter_tokens(bad_ids=stop_ids_technology)

In [16]:
# create corpus
corpus_technology = [dictionary_technology.doc2bow(tokens) for tokens in data_technology]

In [17]:
from collections import Counter

# Count word occurrences
word_counts_technology = Counter()
for doc in corpus_technology:
    for word_id, count in doc:
        word = dictionary_technology[word_id]
        word_counts_technology[word] += count

# Get most frequent words
most_common_technology = word_counts_technology.most_common(100)
print(most_common_technology)

[('app', 39720), ('design', 31958), ('people', 31013), ('device', 30676), ('system', 29730), ('user', 28330), ('year', 27333), ('way', 26500), ('power', 24389), ('world', 24184), ('technology', 23965), ('video', 21718), ('goal', 20748), ('phone', 20672), ('play', 19988), ('part', 18539), ('life', 18421), ('feature', 18172), ('team', 18028), ('development', 17840), ('experience', 17774), ('day', 17587), ('software', 17564), ('data', 17200), ('business', 16974), ('battery', 16912), ('home', 16488), ('prototype', 15725), ('application', 15594), ('support', 15540), ('community', 15178), ('work', 15047), ('idea', 14839), ('cost', 14608), ('production', 14579), ('company', 14483), ('service', 14265), ('board', 14064), ('platform', 14049), ('level', 13721), ('thing', 13459), ('information', 13440), ('reward', 12976), ('tool', 12566), ('quality', 12455), ('game', 12197), ('water', 12163), ('order', 11971), ('friend', 11919), ('student', 11894), ('hand', 11880), ('control', 11869), ('computer',

In [18]:
print('Number of unique tokens: %d' % len(dictionary_technology))
print('Number of documents: %d' % len(corpus_technology))

Number of unique tokens: 19789
Number of documents: 26976


In [16]:
# Initialize an empty list to store the LDA models
lda_models_technology = []

In [19]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_technology, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_technology, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_technology, texts=data_technology, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [20]:
# Iterate over the desired topic counts from 10 to 300 in steps of 10
#for num_topics in tqdm(range(10, 301, 10)):
    # Create an LDA model with the current topic count
lda_model = gensim.models.LdaModel(corpus=corpus_technology, id2word=dictionary_technology, num_topics=100, passes=10, per_word_topics=True, chunksize=1500, iterations=150, alpha='auto', callbacks=callbacks)

    # Save the model to Google Drive
    #model_path = main_path + f"lda_technology/{num_topics}/lda_model_technology_{num_topics}"
    #lda_model.save(model_path)

    # Append the model path to the list
    #lda_models_technology.append(lda_model)

Setting up a new session...


In [44]:
vis = gensimvis.prepare(lda_model, corpus_technology, dictionary_technology)
pyLDAvis.save_html(vis, './LDA/technology/vis/100_POS.html')

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [21]:
lda_model.save('./LDA/technology/100/LDA_technology_100_POS')

In [24]:
lda_technology_50 = gensim.models.LdaModel.load('./LDA/technology/50/LDA_technology_50')
lda_technology_100 = gensim.models.LdaModel.load('./LDA/technology/100/LDA_technology_100')
lda_technology_200 = gensim.models.LdaModel.load('./LDA/technology/200/LDA_technology_200')

In [25]:
vis_50 = gensimvis.prepare(lda_technology_50, corpus_technology, dictionary_technology)
vis_100 = gensimvis.prepare(lda_technology_100, corpus_technology, dictionary_technology)
vis_200 = gensimvis.prepare(lda_technology_200, corpus_technology, dictionary_technology)

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [27]:
pyLDAvis.save_html(vis_50, './LDA/technology/vis/50.html')
pyLDAvis.save_html(vis_100, './LDA/technology/vis/100.html')
pyLDAvis.save_html(vis_200, './LDA/technology/vis/200.html')

In [ ]:
# Loop through the created LDA models
for num_topics, lda_model in zip(range(10, 301, 10), lda_models_technology):
    # Create the visualization
    vis = gensimvis.prepare(lda_model, corpus_technology, dictionary_technology)

    # Save the visualization as an HTML file
    html_path = main_path + f"lda_technology/{num_topics}/lda_vis_technology_{num_topics}.html"
    pyLDAvis.save_html(vis, html_path)

### Games Category

In [22]:
df_lda_kickstarter_games = df_lda_kickstarter.copy()

In [23]:
# Filter Dataframe by Games category (number 12)
df_lda_kickstarter_games = df_lda_kickstarter_games[(df_lda_kickstarter_games['project_category_id'] == 12) | (df_lda_kickstarter_games['project_parent_category_id'] == 12)]

In [24]:
# Preprocessing
df_lda_kickstarter_games['project_description'] = df_lda_kickstarter_games['project_description'].progress_apply(preprocess_text_helper)

  0%|          | 0/39654 [00:00<?, ?it/s]

In [25]:
data_games = df_lda_kickstarter_games['project_description'].tolist()
# Compute bigrams/trigrams
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
ngrams_games = Phrases(data_games, min_count=20)
for idx in range(len(data_games)):
    for token in ngrams_games[data_games[idx]]:
        if '_' in token:
            # Token is a bigram/trigram, add to document.
            data_games[idx].append(token)

In [26]:
# create dictionary
dictionary_games = corpora.Dictionary(data_games)

In [27]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_games.filter_extremes(no_below=5, no_above=0.5)

In [28]:
# Define your custom stoplist
custom_stoplist_games = ["project", "kickstarter", "pledge", "backer", "campaign", "product"] #to be extended
# Add your custom stop words to the dictionary
stop_ids_games = [dictionary_games.token2id[word] for word in custom_stoplist_games if word in dictionary_games.token2id]
# Remove the stop words from the dictionary
dictionary_games.filter_tokens(bad_ids=stop_ids_games)

In [29]:
# create corpus
corpus_games = [dictionary_games.doc2bow(tokens) for tokens in data_games]

In [30]:
from collections import Counter

# Count word occurrences
word_counts_games = Counter()
for doc in corpus_games:
    for word_id, count in doc:
        word = dictionary_games[word_id]
        word_counts_games[word] += count

# Get most frequent words
most_common_games = word_counts_games.most_common(100)
print(most_common_games)

[('card', 187046), ('character', 65596), ('level', 62734), ('deck', 61114), ('book', 48977), ('reward', 48967), ('play', 44231), ('art', 41949), ('design', 41142), ('year', 40574), ('stretch', 39548), ('dice', 38203), ('stretch_goal', 37227), ('people', 35017), ('story', 34738), ('copy', 34333), ('board', 34033), ('rule', 33813), ('adventure', 32497), ('shipping', 32293), ('system', 32043), ('team', 31380), ('box', 30285), ('set', 30063), ('edition', 29871), ('version', 29191), ('item', 29123), ('cost', 28641), ('life', 27998), ('point', 27868), ('friend', 27775), ('order', 27523), ('thing', 27511), ('color', 26079), ('experience', 25413), ('hand', 25394), ('part', 25250), ('work', 25206), ('page', 25084), ('day', 23422), ('power', 23205), ('action', 22846), ('monster', 22786), ('video', 22296), ('print', 22199), ('ability', 21917), ('number', 21794), ('battle', 21661), ('map', 21426), ('idea', 20619), ('support', 20463), ('piece', 20376), ('everyone', 20289), ('fun', 20132), ('hero', 

In [31]:
print('Number of unique tokens: %d' % len(dictionary_games))
print('Number of documents: %d' % len(corpus_games))

Number of unique tokens: 31660
Number of documents: 39654


In [ ]:
# Initialize an empty list to store the LDA models
lda_models_games = []

In [41]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_games, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_games, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_games, texts=data_games, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [32]:
lda_model_70 = gensim.models.LdaModel(corpus=corpus_games, id2word=dictionary_games, num_topics=70, passes=5, per_word_topics=True, chunksize=2000, iterations=150, alpha='auto')

In [67]:
lda_model_100 = gensim.models.LdaModel(corpus=corpus_games, id2word=dictionary_games, num_topics=100, passes=3, per_word_topics=True, chunksize=20000, iterations=150, alpha='auto')

In [68]:
lda_model_125 = gensim.models.LdaModel(corpus=corpus_games, id2word=dictionary_games, num_topics=125, passes=3, per_word_topics=True, chunksize=20000, iterations=150, alpha='auto')

In [45]:
vis = gensimvis.prepare(lda_model_70, corpus_games, dictionary_games)
pyLDAvis.save_html(vis, './LDA/games/vis/70_POS.html')

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [69]:
vis_75 = gensimvis.prepare(lda_model_75, corpus_games, dictionary_games)
vis_100 = gensimvis.prepare(lda_model_100, corpus_games, dictionary_games)
vis_125 = gensimvis.prepare(lda_model_125, corpus_games, dictionary_games)

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [70]:
pyLDAvis.save_html(vis_75, './LDA/games/vis/75.html')
pyLDAvis.save_html(vis_100, './LDA/games/vis/100.html')
pyLDAvis.save_html(vis_125, './LDA/games/vis/125.html')

In [ ]:
# Iterate over the desired topic counts from 10 to 300 in steps of 10
for num_topics in range(10, 301, 10):
    # Create an LDA model with the current topic count
    lda_model = gensim.models.LdaMulticore(corpus=corpus_games, id2word=dictionary_games, num_topics=num_topics, passes=30, workers=8, per_word_topics=True, chunksize=100, iterations=150, eval_every=None, gamma_threshold=0.001)

    # Save the model to Google Drive
    model_path = main_path + f"lda_games/{num_topics}/lda_model_games_{num_topics}"
    lda_model.save(model_path)

    # Append the model path to the list
    lda_models_games.append(lda_model)

In [ ]:
# Loop through the created LDA models
for num_topics, lda_model in zip(range(10, 301, 10), lda_models_games):
    # Create the visualization
    vis = gensimvis.prepare(lda_model, corpus_games, dictionary_games)

    # Save the visualization as an HTML file
    html_path = main_path + f"lda_games/{num_topics}/lda_vis_games_{num_topics}.html"
    pyLDAvis.save_html(vis, html_path)

### Design Category

In [33]:
df_lda_kickstarter_design = df_lda_kickstarter.copy()

In [34]:
# Filter Dataframe by Design category (number 7)
df_lda_kickstarter_design = df_lda_kickstarter_design[(df_lda_kickstarter_design['project_category_id'] == 7) | (df_lda_kickstarter_design['project_parent_category_id'] == 7)]

In [35]:
# Preprocessing general steps
nltk.download('stopwords')
nltk.download('wordnet')
df_lda_kickstarter_design['project_description'] = df_lda_kickstarter_design['project_description'].progress_apply(preprocess_text_helper)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CoolerMaster\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CoolerMaster\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  0%|          | 0/28290 [00:00<?, ?it/s]

In [36]:
data_design = df_lda_kickstarter_design['project_description'].tolist()
# Compute n-grams
from gensim.models import Phrases

# Add n-grams to docs (only ones that appear 20 times or more).
ngrams_design = Phrases(data_design, min_count=20)
for idx in range(len(data_design)):
    for token in ngrams_design[data_design[idx]]:
        if '_' in token:
            # Token is a n-grams, add to document.
            data_design[idx].append(token)

In [37]:
# create dictionary
dictionary_design = corpora.Dictionary(data_design)

In [38]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_design.filter_extremes(no_below=5, no_above=0.5)

In [39]:
# Define your custom stoplist
custom_stoplist_design = ["project", "kickstarter", "pledge", "backer", "campaign", "product"] #to be extended
# Add your custom stop words to the dictionary
stop_ids_design = [dictionary_design.token2id[word] for word in custom_stoplist_design if word in dictionary_design.token2id]
# Remove the stop words from the dictionary
dictionary_design.filter_tokens(bad_ids=stop_ids_design)

In [40]:
# create corpus
corpus_design = [dictionary_design.doc2bow(tokens) for tokens in data_design]

In [41]:
from collections import Counter

# Count word occurrences
word_counts_design = Counter()
for doc in corpus_design:
    for word_id, count in doc:
        word = dictionary_design[word_id]
        word_counts_design[word] += count

# Get most frequent words
most_common_design = word_counts_design.most_common(100)
print(most_common_design)

[('year', 29868), ('way', 26199), ('production', 25489), ('color', 25434), ('material', 23784), ('goal', 23654), ('hand', 22765), ('people', 21379), ('day', 21053), ('reward', 20921), ('quality', 20735), ('bag', 20164), ('prototype', 19939), ('world', 19687), ('life', 19384), ('case', 17866), ('order', 17849), ('water', 17793), ('part', 17763), ('play', 17668), ('size', 17603), ('idea', 16367), ('card', 16111), ('support', 15584), ('work', 15074), ('process', 14131), ('home', 14068), ('tool', 13981), ('system', 13738), ('thing', 13736), ('cost', 13113), ('company', 12783), ('pocket', 12689), ('use', 12287), ('phone', 12187), ('experience', 11951), ('option', 11928), ('friend', 11918), ('piece', 11869), ('place', 11760), ('space', 11305), ('video', 11272), ('something', 11270), ('help', 11210), ('steel', 11046), ('business', 10898), ('market', 10878), ('team', 10829), ('level', 10716), ('device', 10470), ('wallet', 10402), ('family', 10353), ('watch', 10252), ('line', 10079), ('shipping

In [42]:
print('Number of unique tokens: %d' % len(dictionary_design))
print('Number of documents: %d' % len(corpus_design))

Number of unique tokens: 21627
Number of documents: 28290


In [ ]:
# Initialize an empty list to store the LDA models
lda_models_design = []

In [ ]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_design, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_design, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_design, texts=data_design, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [81]:
lda_model_75 = gensim.models.LdaModel(corpus=corpus_design, id2word=dictionary_design, num_topics=75, passes=3, per_word_topics=True, chunksize=15000, iterations=150, alpha='auto')

In [43]:
lda_model_100 = gensim.models.LdaModel(corpus=corpus_design, id2word=dictionary_design, num_topics=100, passes=5, per_word_topics=True, chunksize=1500, iterations=150, alpha='auto')

In [83]:
lda_model_125 = gensim.models.LdaModel(corpus=corpus_design, id2word=dictionary_design, num_topics=125, passes=3, per_word_topics=True, chunksize=15000, iterations=150, alpha='auto')

In [46]:
vis = gensimvis.prepare(lda_model_100, corpus_design, dictionary_design)
pyLDAvis.save_html(vis, './LDA/design/vis/100_POS.html')

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [84]:
vis_75 = gensimvis.prepare(lda_model_75, corpus_design, dictionary_design)
vis_100 = gensimvis.prepare(lda_model_100, corpus_design, dictionary_design)
vis_125 = gensimvis.prepare(lda_model_125, corpus_design, dictionary_design)

d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
d:\anaconda3\envs\Bachelorthesis\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [85]:
pyLDAvis.save_html(vis_75, './LDA/design/vis/75.html')
pyLDAvis.save_html(vis_100, './LDA/design/vis/100.html')
pyLDAvis.save_html(vis_125, './LDA/design/vis/125.html')

In [ ]:
# Iterate over the desired topic counts from 10 to 300 in steps of 10
for num_topics in tqdm(range(10, 301, 10)):
    # Create an LDA model with the current topic count
    lda_model = gensim.models.LdaMulticore(corpus=corpus_design, id2word=dictionary_design, num_topics=num_topics, passes=30, workers=8, per_word_topics=True, chunksize=100, iterations=150, eval_every=None, gamma_threshold=0.001)

    # Save the model to Google Drive
    model_path = main_path + f"lda_design/{num_topics}/lda_model_design_{num_topics}"
    lda_model.save(model_path)

    # Append the model path to the list
    lda_models_design.append(lda_model)

In [ ]:
# Loop through the created LDA models
for num_topics, lda_model in zip(range(10, 301, 10), lda_models_design):
    # Create the visualization
    vis = gensimvis.prepare(lda_model, corpus_design, dictionary_design)

    # Save the visualization as an HTML file
    html_path = main_path + f"lda_design/{num_topics}/lda_vis_design_{num_topics}.html"
    pyLDAvis.save_html(vis, html_path)